In [ ]:
import tkinter as tk
from tkinter import scrolledtext, filedialog
from collections import Counter
import heapq
import json


class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


class HuffmanTree:
    def __init__(self, frequencies):
        self.root = self.build_tree(frequencies)
        self.codes = {}
        self.reverse_codes = {}
        self.generate_codes(self.root, "")

    def build_tree(self, frequencies):
        heap = [Node(char, freq) for char, freq in frequencies.items()]
        heapq.heapify(heap)

        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            merged = Node(None, left.freq + right.freq)
            merged.left = left
            merged.right = right
            heapq.heappush(heap, merged)

        return heap[0]

    def generate_codes(self, node, current_code):
        if node is not None:
            if node.char is not None:
                self.codes[node.char] = current_code
                self.reverse_codes[current_code] = node.char
            self.generate_codes(node.left, current_code + "0")
            self.generate_codes(node.right, current_code + "1")

    def encode(self, text):
        encoded_output = ""
        for char in text:
            encoded_output += self.codes[char]
        return encoded_output

    def decode(self, encoded_text):
        current_code = ""
        decoded_output = ""
        for bit in encoded_text:
            current_code += bit
            if current_code in self.reverse_codes:
                decoded_output += self.reverse_codes[current_code]
                current_code = ""
        return decoded_output


def encode_text_from_file():
    file_path = filedialog.askopenfilename(
        title="Select a File",
        filetypes=[("Text Files", "*.txt")]
    )
    if not file_path:
        return

    with open(file_path, 'r', encoding='utf-8') as file:
        text_to_encode = file.read().strip()

    if not text_to_encode:
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, "The selected file is empty.")
        return

    frequencies = Counter(text_to_encode)
    huffman_tree = HuffmanTree(frequencies)
    encoded_text = huffman_tree.encode(text_to_encode)

    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, "Original Text:\n" + text_to_encode + "\n\nCharacter Codes:\n")
    for char, code in huffman_tree.codes.items():
        output_text.insert(tk.END, f"'{char}': {code}\n")

    output_text.insert(tk.END, "\nEncoded Text:\n" + encoded_text)

    # Save the encoded text and Huffman codes to a file
    save_encoded_text(encoded_text, huffman_tree.codes)


def save_encoded_text(encoded_text, codes):
    save_path = filedialog.asksaveasfilename(
        title="Save Compressed File",
        defaultextension=".json",
        filetypes=[("JSON Files", "*.json"), ("All Files", "*.*")]
    )
    if not save_path:
        return

    # Save both encoded text and Huffman codes
    with open(save_path, 'w', encoding='utf-8') as file:
        json.dump({"encoded_text": encoded_text, "codes": codes}, file)


def decode_text_from_file():
    file_path = filedialog.askopenfilename(
        title="Select a Compressed File",
        filetypes=[("JSON Files", "*.json")]
    )
    if not file_path:
        return

    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    encoded_text = data.get("encoded_text", "")
    codes = data.get("codes", {})

    if not encoded_text:
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, "The selected file is empty.")
        return

    huffman_tree = HuffmanTree(Counter(codes.keys()))  # Rebuild the tree using the codes
    huffman_tree.codes = codes
    huffman_tree.reverse_codes = {v: k for k, v in codes.items()}  # Reverse the codes for decoding
    decoded_text = huffman_tree.decode(encoded_text)

    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, "Decoded Text:\n" + decoded_text)


# Set up the main application window
root = tk.Tk()
root.title("Huffman Encoding and Decoding GUI")

# File upload button for encoding
upload_button = tk.Button(root, text="Upload Text File to Encode", command=encode_text_from_file)
upload_button.pack(pady=10)

# File upload button for decoding
decode_button = tk.Button(root, text="Upload Compressed File to Decode", command=decode_text_from_file)
decode_button.pack(pady=10)

# Output text area - increased size
output_text = scrolledtext.ScrolledText(root, width=80, height=25)  # Adjusted width and height
output_text.pack(pady=10)

# Start the GUI event loop
root.mainloop()
